In [1]:
from torch import nn
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.models.bert.modeling_bert import BertModel, BertPreTrainedModel

In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
import sklearn
import pandas as pd
import numpy as np

In [4]:
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoConfig

In [5]:
import os
from os import listdir
import sys
import json
from os import path

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [7]:
from sklearn.model_selection import KFold

In [8]:
models_dir = "models/bert-k-fold"

In [9]:
import pandas as pd
labels = open('data/classes.txt').read().splitlines()
df = pd.read_csv("data/belief_benchmark_all_train.csv")

In [10]:
df.head()

,text,sentiment,label
0,Private-sector actors expect their business pa...,NEGATIVE,1
1,Lack of product uniformity is perceived by con...,"NEGATIVE, NEGATIVE, NEGATIVE, NEGATIVE",1
2,"However , to the extent that the empirical str...",POSITIVE,2
3,Firms prefer to contract with producers with w...,POSITIVE,2
4,Farmers attempted to use alternate wetting and...,POSITIVE,2


In [11]:
transformer_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(transformer_name)
# NOTE: for cross validation, the model should be initialized inside the cv loop

In [12]:
def tokenize(batch):
    return tokenizer(batch['text'], truncation=True)

In [13]:
def compute_metrics(eval_pred):
    y_true = eval_pred.label_ids
    y_pred = np.argmax(eval_pred.predictions, axis=-1)
    report = metrics.classification_report(y_true, y_pred)
    print("report: \n", report)
    
    print("rep type: ", type(report))
    

    return {'f1':metrics.f1_score(y_true, y_pred, average="macro")}

In [14]:
# Note: not used right now, but can be
# https://github.com/huggingface/transformers/blob/65659a29cf5a079842e61a63d57fa24474288998/src/transformers/models/bert/modeling_bert.py#L1486

class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            **kwargs,
        )
        cls_outputs = outputs.last_hidden_state[:, 0, :]
        cls_outputs = self.dropout(cls_outputs)
        logits = self.classifier(cls_outputs)
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


In [15]:
# this is for creating cross-validation folds
def get_sample_based_on_idx(data, indeces):
    return data.iloc[indeces, :].reset_index()

In [16]:
# defining hyperparams
num_epochs = 8
batch_size = 6
weight_decay = 0.01
print(f"num_epochs: {num_epochs}, batch_size: {batch_size}, weight_decay: {weight_decay}")
training_args = TrainingArguments(
    output_dir="./results_triggerless", # is this location in the tmp dir? 
    log_level='error',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    weight_decay=weight_decay,
    load_best_model_at_end=True, # this is supposed to make sure the best model is loaded by the trainer at the end
    metric_for_best_model="eval_f1" 
    )

In [17]:
# specify what percent of the all_train data should be used in cross-fold validation 
# we can use this to create a learning curve, e.g., performance with 50, 80, 90% of data

from sklearn.model_selection import train_test_split
train_size = 0.8

if train_size < 1.0:
    df, _ = train_test_split(df, test_size=1.0-train_size, random_state=1, stratify=df[['label']])
print(f"train_size: {train_size} (total samples: {len(df)})")

In [18]:
output = open("original.txt", "a")

fold = 0
kfold = KFold(n_splits=5, shuffle=True, random_state=1)
for train_df_idx, eval_df_idx in kfold.split(df):
    
    print("FOLD: ", fold)
    output.write(f"FOLD: {fold}\n")
    new_df = pd.DataFrame()
    
    train_df = get_sample_based_on_idx(df, train_df_idx)
    print("LEN DF: ", len(train_df))
    output.write(f"LEN DF: {len(train_df)}\n")
#     train_df['label'] = [int(item) for item in train_df["annotation: b (belief or attitude), n (not a belief and not an attitude)"]]
    print("done train df")
    output.write("done train df\n")
    eval_df = get_sample_based_on_idx(df, eval_df_idx)
#     eval_df["label"] = [int(item) for item in eval_df['annotation: b (belief or attitude), n (not a belief and not an attitude)']]
    print("done eval df")
    output.write("done eval df\n")
    print("LEN EVAL: ", len(eval_df))
    output.write(f"LEN EVAL: {len(eval_df)}\n")
#     print(eval_df.head())
    ds = DatasetDict()
    ds['train'] = Dataset.from_pandas(train_df)
    ds['validation'] = Dataset.from_pandas(eval_df)
    train_ds = ds['train'].map(
        tokenize, batched=True,
        # remove_columns=['index', 'sentence', 'trigger', 'annotation: b (belief or attitude), n (not a belief and not an attitude)', 'paragraph'],
    )
    eval_ds = ds['validation'].map(
        tokenize,
        batched=True,
        # remove_columns=['index', 'sentence', 'trigger', 'annotation: b (belief or attitude), n (not a belief and not an attitude)', 'paragraph'],
    )

#     config = AutoConfig.from_pretrained(
#         transformer_name,
#         num_labels=2,
#     )

    model = AutoModelForSequenceClassification.from_pretrained(transformer_name, num_labels=4)
    tokenizer = AutoTokenizer.from_pretrained(transformer_name)
#     model = (
#         BertForSequenceClassification
#         .from_pretrained(transformer_name, config=config)
#     )
    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        tokenizer=tokenizer,
    )
    trainer.train()
    # after training, predict (will use best model?)
    preds = trainer.predict(eval_ds)
#     print("HERE: " , preds)
    final_preds = [np.argmax(x) for x in preds.predictions]
    real_f1 = metrics.f1_score(final_preds, eval_df["label"], average="macro")
    print("F-1: ", real_f1)
    output.write(f"F-1: {real_f1}\n")
    model_name = f"{transformer_name}-best-of-fold-{fold}-f1-{real_f1}"
    model_dir = os.path.join(models_dir, model_name)

    trainer.save_model(model_dir)

    for i, item in enumerate(final_preds):
        if item != eval_ds["label"][i]:
            wrong_df = pd.DataFrame()
            wrong_df["text"] = [eval_df["text"][i]]
            wrong_df["real"] = [eval_df["label"][i]]
            wrong_df["predicted"] = [item]
            new_df = pd.concat([new_df, wrong_df])

    new_df.to_csv(f"{models_dir}/wrong_predictions_{fold}.csv")
    fold += 1
        

FOLD:  0
LEN DF:  138
done train df
done eval df
LEN EVAL:  35


Map:   0%|          | 0/138 [00:00<?, ? examples/s]

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/184 [00:00<?, ?it/s]

...

c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


report: 
               precision    recall  f1-score   support

           0       0.69      0.60      0.64        15
           1       0.57      0.67      0.62         6
           2       0.50      0.78      0.61         9
           3       0.00      0.00      0.00         4

    accuracy                           0.59        34
   macro avg       0.44      0.51      0.47        34
weighted avg       0.54      0.59      0.55        34

rep type:  <class 'str'>
{'eval_loss': 1.5565199851989746, 'eval_f1': 0.4667343526039178, 'eval_runtime': 0.2634, 'eval_samples_per_second': 129.061, 'eval_steps_per_second': 22.775, 'epoch': 8.0}
{'train_runtime': 62.9142, 'train_samples_per_second': 17.675, 'train_steps_per_second': 3.052, 'train_loss': 0.4454421599706014, 'epoch': 8.0}


  0%|          | 0/6 [00:00<?, ?it/s]

c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


report: 
               precision    recall  f1-score   support

           0       0.63      0.80      0.71        15
           1       0.75      0.50      0.60         6
           2       0.55      0.67      0.60         9
           3       0.00      0.00      0.00         4

    accuracy                           0.62        34
   macro avg       0.48      0.49      0.48        34
weighted avg       0.56      0.62      0.58        34

rep type:  <class 'str'>
F-1:  0.4764705882352941


In [19]:
holdout_df = df = pd.read_csv("data/belief_benchmark_holdout.csv")
holdout_ds = Dataset.from_pandas(holdout_df)
holdout_ds = holdout_ds.map(tokenize, batched=True)

trainer.predict(holdout_ds)

Map:   0%|          | 0/146 [00:00<?, ? examples/s]

  0%|          | 0/25 [00:00<?, ?it/s]

report: 
               precision    recall  f1-score   support

           0       0.53      0.80      0.63        49
           1       0.67      0.38      0.49        26
           2       0.65      0.63      0.64        59
           3       0.00      0.00      0.00        12

    accuracy                           0.59       146
   macro avg       0.46      0.45      0.44       146
weighted avg       0.56      0.59      0.56       146

rep type:  <class 'str'>


c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PredictionOutput(predictions=array([[ 3.12748075e+00, -1.11327565e+00, -7.41954386e-01,
        -1.59095049e+00],
       [ 2.39527154e+00, -1.87988591e+00,  1.11360741e+00,
        -1.95397151e+00],
       [-2.22187090e+00, -1.84547091e+00,  4.52050304e+00,
        -1.39935017e+00],
       [ 3.85241342e+00, -1.50445819e+00, -1.24845469e+00,
        -1.21730661e+00],
       [ 2.42610884e+00, -2.10561609e+00,  9.97956753e-01,
        -1.43785489e+00],
       [ 7.10558712e-01,  2.01579595e+00, -2.03670073e+00,
        -7.05665588e-01],
       [ 2.23595381e+00,  1.01800799e+00, -2.00976968e+00,
        -1.51437628e+00],
       [-1.18690062e+00, -2.12582541e+00,  4.07473803e+00,
        -1.74679625e+00],
       [ 2.16095924e+00, -1.74904287e+00,  1.31997478e+00,
        -1.46853054e+00],
       [-2.51119757e+00, -1.46990967e+00,  4.16050053e+00,
        -6.10300362e-01],
       [-7.23505914e-01, -2.18629956e+00,  2.73375130e+00,
        -1.09689020e-01],
       [ 3.44948816e+00,  2.81788707

In [20]:
import torch
torch.cuda.empty_cache()

In [21]:
torch.cuda.memory_summary(device=None, abbreviated=False)
output.write("{torch.cuda.memory_summary(device=None, abbreviated=False)}")

output.close()